In [1]:
%matplotlib inline
import numpy as np

from transform import combine_tsv_files, load_process_and_store_spreadsheets

In [2]:
from transform import load_avenio_files

In [ ]:
def harmonic_mean(t0, t1):
    return t0 * t1 / (t0 + t1)

load_process_and_store_spreadsheets(
    transformation=harmonic_mean,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

> /home/hylke/workspace/avenio/transform.py(355)load_process_and_store_spreadsheets()
    354         # combined with `transformation`.
--> 355         transposed_sheets = transpose_and_transform(
    356             clean_mutation_sheet, column_pair, transformation

ipdb> n
> /home/hylke/workspace/avenio/transform.py(356)load_process_and_store_spreadsheets()
    355         transposed_sheets = transpose_and_transform(
--> 356             clean_mutation_sheet, column_pair, transformation
    357         )

ipdb> 
> /home/hylke/workspace/avenio/transform.py(355)load_process_and_store_spreadsheets()
    354         # combined with `transformation`.
--> 355         transposed_sheets = transpose_and_transform(
    356             clean_mutation_sheet, column_pair, transformation

ipdb> 
> /home/hylke/workspace/avenio/transform.py(358)load_process_and_store_spreadsheets()
    357         )
--> 358         timepoint_names = ("t0", "t1", transformation.__name__)
    359 

ipdb> p transposed_s

In [3]:
def difference(t0, t1):
    return t1 - t0

load_process_and_store_spreadsheets(
    transformation=difference,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

In [4]:
def relative_difference(t0, t1):
    if t0 != 0.0:
        return (t1-t0)/t0
    return (t1 - t0)/t1

load_process_and_store_spreadsheets(
    transformation=relative_difference,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

In [5]:
def up_or_down(t0, t1):
    return np.sign(t1 - t0)


load_process_and_store_spreadsheets(
    transformation=up_or_down,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

In [6]:
def margin_sign(t0, t1, margin=0.15):
    if t0 == 0.0:  # Complete clearance at baseline.
        return 1
    if t1 == 0.0:  # Complete clearance at follow up.
        return -1
    dt = (t1-t0)/t0
    if dt > margin:
        return 1
    elif dt < -margin:
        return -1
    return 0


load_process_and_store_spreadsheets(
    transformation=margin_sign,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

Data without train/test split, with CNV scores merged in.

In [7]:
def to_excel(snv_name, cnv_name):
    excel_filename = snv_name.replace('.tsv', '_CNV.xlsx')
    X, y = combine_tsv_files(snv_name, cnv_name)
    X.merge(y, left_index=True, right_index=True).to_excel(excel_filename)
    print(f'Wrote {excel_filename}.')

In [8]:
for coarseness in ('gene', 'pathway'):
    for transformation in ('harmonic_mean', 'difference', 'relative_difference', 'up_or_down', 'margin_sign'):
        to_excel(
            f"output/all__{coarseness}__{transformation}__Allele Fraction.tsv",
            f"output/all__{coarseness}__{transformation}__CNV Score.tsv",
        )
        to_excel(
            f"output/all__{coarseness}__{transformation}__No. Mutant Molecules per mL.tsv",
            f"output/all__{coarseness}__{transformation}__CNV Score.tsv",
        )

Wrote output/all__gene__harmonic_mean__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__harmonic_mean__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__gene__difference__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__difference__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__gene__relative_difference__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__relative_difference__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__gene__up_or_down__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__up_or_down__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__gene__margin_sign__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__margin_sign__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__pathway__harmonic_mean__Allele Fraction_CNV.xlsx.
Wrote output/all__pathway__harmonic_mean__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__pathway__difference__Allele Fraction_CNV.xlsx.
Wrote output/all__pathway__difference__No. Mutant Molecules per mL_CNV.xlsx.
Wrot